# Voxel-Fill_Holes Script

## Post-processing protocol for filling holes within a label layer of a specific size - Developed by Avery Pennington (RFI), edited by Sam Kersley (RFI) and Dolapo Adebo (RFI).

#### This script is for finding holes of a specific size, either in a 2D or 3D format, and then creating an infilled mask

### To run this notebook, you must have the following packages installed within the virtual environemnt you have opened this file within; *numpy, scipy, h5py, scikit-image, matplotlib and tifffile* (version *2022.8.12* or better). If a 'no-module' error is found, install the required module as neccisary.

- Futher information reguarding the MONAI package can be found at 'https://github.com/Project-MONAI/MONAI'

In [ ]:
#If your environment is lacking the required packages;
pip install numpy
pip install scipy
pip install h5py
pip install scikit-image
pip install matplotlib
pip install tifffile==2022.8.12

### EXAMPLE_CODE; Code Example and illistrated notes;
#### Do not run, only for explanation of code utility and functionality.

In [ ]:
%matplotlib inline 
%load_ext autoreload
%autoreload 2

import numpy as np
import os
from matplotlib import pyplot as plt

plt.rc('image', cmap='gray') 
plt.rcParams["axes.grid"] = False 

The first cell delineated the inPuts needed to run the data analysis cleanly 

*%matplotlib inline/ %load_ext autoreload/ %autoreload 2*

- IPython "magic commands" used in Jupyter Notebook to enhance tworkflow when working with Python.
- 'matplotlib inline' displays Matplotlib plots directly inside the Jupyter Notebook and presented as images; better for visualisation.
- 'load_ext autoreload' loads the autoreload extension in IPython where; 
- 'autoreload 2' can then be used to reload the used imported modules before executing complex code; better for the jupyter kernel.

*import numpy as np/ import os/ from matplotlib import pyplot as plt*

- Importing the requied modiules to run the script; numpy, os and matplotlib (plt)

*plt.rc('image', cmap='gray')/ plt.rcParams["axes.grid"] = False*

- Edits the matplotlib package with overarching commands; set the aimges to greyscals and produce plots with no gridlines. 

In [ ]:
from os import listdir 
from os.path import isfile, join
import glob

imgs_path = '/ceph/users/tor38477/WLIU_GUAC-Work/IMAGES-LABELS-TIFF_FILES/06_25-07_25' #Loads all image files within this directory; personal directory - wixi8809/Work; SHOULD BE CHANGED DEPENDING ON THE ROOT DIRECTORY NEEDED
fullnames = glob.glob(join(imgs_path,"*.h5")) 
names = [(os.path.basename(n)[11:], n) for n in fullnames] 
names.sort()
names #Proccessing the information into a new list; cleaning it up for further processing 

len(names)

This cell allows you to choose the file you wish to edit. It allows you to pick a directory within your user space and list the files availible to be read. 

*from os import listdir/ from os.path import isfile, join/ import glob*

- Import the required packaged to select and read the image files.

*img_path* 

- Asigns the directory you wish the notebook to read and list the files within; This should be the full directory path e.g. /ceph/users/ etc.

*fullnames/ names/ names.sort* 

- You can use these functions to sort the files into alphabetical order creating a list that can then be chosen from by the notebook. executing the *'names'* function will then display the availible files as an output.

Once output, the next small cell can then be used to list the number of availible files as an integer; *len(names)*

In [ ]:
sel_names = names[0:1] #Select the specific image you wish to work with (E.G. [0:1] is the first file name)
sel_names 

This cell will let you choose a file from the directory and assign it to the attribute sel_names.

- To select a file from a list, you will be required to designate a file using the square brackets. The files will be ordered by the previous cell, hence you must find the number of the required file situated within this list. the first numebred location will have the designation range of 0:1, the next will be 1:2 and so on (the first numebr in the range will be the starting number in the list, 0 is the first, second is 1 etc.)
- It is best to run 1 file at a time to not put strain on the notebook kernal and avoid overwriting of similar filenames; mutiple files can be run in the same notebook however in different batches; run all 9 cells per file you wish to process.

#### There are 2 versions of this code; one for the 3D method and one for the 2D method. You require all fo the cells until this point to make either varient executable, however from this point each method id different; 3D requiring 2 cells, and 2D requiring 3 cells. 

In [ ]:
#3D package inputs.
import h5py
import tifffile

The cell delineated the main inputs needed to calculate and output the 3D fill-holes protocol.

*import h5py/ import tifffile*

- Importing the requied modules to run the script; h5py and tifffile

In [ ]:
#3D calculation and output.
for i, (basename, fullname) in enumerate(sel_names):
    with tifffile.TiffFile(fullname) as tif:
        img = tif.asarray().copy()
        fname = os.path.basename(fullname)[:-3]
        structure=np.ones(('Voxel_Volume')) # Structure = Volume of 3 sides; hole-size to fill represented as 'W, H, L'
        img_crop_fh = ndimage.binary_fill_holes(img, structure) * 1
        tifffile.imwrite(fname + '_crop_fh.tif', img_crop_fh.astype(np.uint8))
        plt.figure()
        plt.imshow(img_crop_fh[30,:])
        del img

#Ennd of 3D protocol

This cell makes a copy of the label data as an array, and uses *ndimage* to search for holes of a specific size designated by the *'Voxel-Volume'*; the 'voxel-volume' is expressed as a 3-integer co-ordinate designation, where each integer must be the same representing a cube e.g. 5^3 is 5, 5, 5 (W, H, L)

- (5, 5, 5) has proven to be the best starting Voxel-Volume. 
- The cell then takes this new mask and creates a new .tif file with the holes of the size or smaller infilled. 

In [ ]:
#2D package inputs.
import h5py
import tifffile
import os 
from skimage import morphology

The cell delineated the main inputs needed to calculate and output the 2D fill-holes protocol.

*import h5py/ import tifffile/ import os/ from skimage import morphology*

- Importing the requied modules to run the script; h5py, tifffile and morphology from skimage

In [ ]:
#3D Voxel-Volume designation.
HOLE_SIZE='Integer' #Designate the Hole-Size; square of thr voxel input; size=64 will remove the holes of size 8 squared

This cell designates the size of the 2D fill-holes protocol serach; the 'integer' should be the square of the voxel sixe you require e.g. voxel=8: 64='integer'

- 49 (7^2) has proven to be the best starting HOLE_SIZE. 

In [ ]:
#2D calculation and output.
##Run scan crop and finding protocol for each slice; [0], [1], [2] are the 3 slice directions
img_crops = []
for i, (basename, fullname) in enumerate(sel_names):
    with tifffile.TiffFile(fullname) as tif:
        img = tif.asarray().copy()
        for j in range(img.shape[0]):
            s = img[j,:]
            sfh = morphology.remove_small_holes(s, HOLE_SIZE).astype(int)
            img[j,:] = sfh
        for j in range(img.shape[1]):
            s = img[:,j,:]
            sfh = morphology.remove_small_holes(s, HOLE_SIZE).astype(int)
            img[:,j,:] = sfh
        for j in range(img.shape[2]):
            s = img[:,:,j]
            sfh = morphology.remove_small_holes(s, HOLE_SIZE).astype(int)
            img[:,:,j] = sfh
        img_out = img
        fname = os.path.basename(fullname)[:-4]
        print(f"Outputting: {fname}")
        outname = os.path.join(fname + '_fh' + str(HOLE_SIZE) + '.tif')
        tifffile.imwrite(outname, img_out.astype(np.uint8))
        plt.figure()
        plt.imshow(img_out[30,:])
        del img

#End of 2D protocol

This cell makes a copy of the label data as an array and searches for and infills the holes as designated by the *HOLE_SIZE* using the *'remove_small_holes'* command from *morphology*. It does this along each axis in turn starting with Z, then Y and finally X. 

- The cell then takes this new mask and creates a new .tif file with the holes of the size or smaller infilled. 

## RUNABLE CELL BATHCES:

These cell batches are duplicatable and can be run in tendem or singularly. 

### DATE; ##.##.## PURPOSE; ~~

#### BLANK-3D-var ##.##.## _work['~~'] - DESC.~~~

In [ ]:
%matplotlib inline 
%load_ext autoreload
%autoreload 2

import numpy as np
import os
from matplotlib import pyplot as plt

plt.rc('image', cmap='gray') 
plt.rcParams["axes.grid"] = False 

In [ ]:
from os import listdir 
from os.path import isfile, join
import glob

imgs_path = '/ceph/users/tor38477/WLIU_GUAC-Work/IMAGES-LABELS-TIFF_FILES/06_25-07_25' #Loads all image files within this directory; personal directory - wixi8809/Work; SHOULD BE CHANGED DEPENDING ON THE ROOT DIRECTORY NEEDED
fullnames = glob.glob(join(imgs_path,"*.h5")) 
names = [(os.path.basename(n)[11:], n) for n in fullnames] 
names.sort()
names #Proccessing the information into a new list; cleaning it up for further processing 

len(names)

In [ ]:
sel_names = names[0:1] #Select the specific image you wish to work with (E.G. [0:1] is the first file name)
sel_names 

In [ ]:
import h5py
import tifffile

In [ ]:
for i, (basename, fullname) in enumerate(sel_names):
    with tifffile.TiffFile(fullname) as tif:
        img = tif.asarray().copy()
        fname = os.path.basename(fullname)[:-3]
        structure=np.ones(('Voxel_Volume')) # Structure = Volume of 3 sides; hole-size to fill represented as 'W, H, L'
        img_crop_fh = ndimage.binary_fill_holes(img, structure) * 1
        tifffile.imwrite(fname + '_crop_fh.tif', img_crop_fh.astype(np.uint8))
        plt.figure()
        plt.imshow(img_crop_fh[30,:])
        del img

#### BLANK-3D-var ##.##.## _work['~~'] - DESC.~~~

In [ ]:
%matplotlib inline 
%load_ext autoreload
%autoreload 2

import numpy as np
import os
from matplotlib import pyplot as plt

plt.rc('image', cmap='gray') 
plt.rcParams["axes.grid"] = False 

In [ ]:
from os import listdir 
from os.path import isfile, join
import glob

imgs_path = '/ceph/users/tor38477/WLIU_GUAC-Work/IMAGES-LABELS-TIFF_FILES/06_25-07_25' #Loads all image files within this directory; personal directory - wixi8809/Work; SHOULD BE CHANGED DEPENDING ON THE ROOT DIRECTORY NEEDED
fullnames = glob.glob(join(imgs_path,"*.h5")) 
names = [(os.path.basename(n)[11:], n) for n in fullnames] 
names.sort()
names #Proccessing the information into a new list; cleaning it up for further processing 

len(names)

In [ ]:
sel_names = names[0:1] #Select the specific image you wish to work with (E.G. [0:1] is the first file name)
sel_names 

In [ ]:
import h5py
import tifffile

In [ ]:
for i, (basename, fullname) in enumerate(sel_names):
    with tifffile.TiffFile(fullname) as tif:
        img = tif.asarray().copy()
        fname = os.path.basename(fullname)[:-3]
        structure=np.ones(('Voxel_Volume')) # Structure = Volume of 3 sides; hole-size to fill represented as 'W, H, L'
        img_crop_fh = ndimage.binary_fill_holes(img, structure) * 1
        tifffile.imwrite(fname + '_crop_fh.tif', img_crop_fh.astype(np.uint8))
        plt.figure()
        plt.imshow(img_crop_fh[30,:])
        del img

#### BLANK-2D-var ##.##.## _work['~~'] - DESC.~~~

In [ ]:
%matplotlib inline 
%load_ext autoreload
%autoreload 2

import numpy as np
import os
from matplotlib import pyplot as plt

plt.rc('image', cmap='gray') 
plt.rcParams["axes.grid"] = False 

In [ ]:
from os import listdir 
from os.path import isfile, join
import glob

imgs_path = '/ceph/users/tor38477/WLIU_GUAC-Work/IMAGES-LABELS-TIFF_FILES/06_25-07_25' #Loads all image files within this directory; personal directory - wixi8809/Work; SHOULD BE CHANGED DEPENDING ON THE ROOT DIRECTORY NEEDED
fullnames = glob.glob(join(imgs_path,"*.h5")) 
names = [(os.path.basename(n)[11:], n) for n in fullnames] 
names.sort()
names #Proccessing the information into a new list; cleaning it up for further processing 

len(names)

In [ ]:
sel_names = names[0:1] #Select the specific image you wish to work with (E.G. [0:1] is the first file name)
sel_names 

In [ ]:
import h5py
import tifffile
import os 
from skimage import morphology

In [ ]:
HOLE_SIZE='Integer' #Designate the Hole-Size; square of thr voxel input; size=64 will remove the holes of size 8 squared

In [ ]:
img_crops = []
for i, (basename, fullname) in enumerate(sel_names):
    with tifffile.TiffFile(fullname) as tif:
        img = tif.asarray().copy()
        for j in range(img.shape[0]):
            s = img[j,:]
            sfh = morphology.remove_small_holes(s, HOLE_SIZE).astype(int)
            img[j,:] = sfh
        for j in range(img.shape[1]):
            s = img[:,j,:]
            sfh = morphology.remove_small_holes(s, HOLE_SIZE).astype(int)
            img[:,j,:] = sfh
        for j in range(img.shape[2]):
            s = img[:,:,j]
            sfh = morphology.remove_small_holes(s, HOLE_SIZE).astype(int)
            img[:,:,j] = sfh
        img_out = img
        fname = os.path.basename(fullname)[:-4]
        print(f"Outputting: {fname}")
        outname = os.path.join(fname + '_fh' + str(HOLE_SIZE) + '.tif')
        tifffile.imwrite(outname, img_out.astype(np.uint8))
        plt.figure()
        plt.imshow(img_out[30,:])
        del img

#### BLANK-2D-var ##.##.## _work['~~'] - DESC.~~~

In [ ]:
%matplotlib inline 
%load_ext autoreload
%autoreload 2

import numpy as np
import os
from matplotlib import pyplot as plt

plt.rc('image', cmap='gray') 
plt.rcParams["axes.grid"] = False 

In [ ]:
from os import listdir 
from os.path import isfile, join
import glob

imgs_path = '/ceph/users/tor38477/WLIU_GUAC-Work/IMAGES-LABELS-TIFF_FILES/06_25-07_25' #Loads all image files within this directory; personal directory - wixi8809/Work; SHOULD BE CHANGED DEPENDING ON THE ROOT DIRECTORY NEEDED
fullnames = glob.glob(join(imgs_path,"*.h5")) 
names = [(os.path.basename(n)[11:], n) for n in fullnames] 
names.sort()
names #Proccessing the information into a new list; cleaning it up for further processing 

len(names)

In [ ]:
sel_names = names[0:1] #Select the specific image you wish to work with (E.G. [0:1] is the first file name)
sel_names 

In [ ]:
import h5py
import tifffile
import os 
from skimage import morphology

In [ ]:
HOLE_SIZE='Integer' #Designate the Hole-Size; square of thr voxel input; size=64 will remove the holes of size 8 squared

In [ ]:
img_crops = []
for i, (basename, fullname) in enumerate(sel_names):
    with tifffile.TiffFile(fullname) as tif:
        img = tif.asarray().copy()
        for j in range(img.shape[0]):
            s = img[j,:]
            sfh = morphology.remove_small_holes(s, HOLE_SIZE).astype(int)
            img[j,:] = sfh
        for j in range(img.shape[1]):
            s = img[:,j,:]
            sfh = morphology.remove_small_holes(s, HOLE_SIZE).astype(int)
            img[:,j,:] = sfh
        for j in range(img.shape[2]):
            s = img[:,:,j]
            sfh = morphology.remove_small_holes(s, HOLE_SIZE).astype(int)
            img[:,:,j] = sfh
        img_out = img
        fname = os.path.basename(fullname)[:-4]
        print(f"Outputting: {fname}")
        outname = os.path.join(fname + '_fh' + str(HOLE_SIZE) + '.tif')
        tifffile.imwrite(outname, img_out.astype(np.uint8))
        plt.figure()
        plt.imshow(img_out[30,:])
        del img